<a href="https://colab.research.google.com/github/israGUT1ERR3Z/FootLab/blob/main/botTelegram_SegmentacionyClasificacionBUENO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytelegrambotapi

In [ ]:
import telebot, imageio, requests, threading, torch, cv2, helper #, time

from PIL import Image
from io import BytesIO
from sklearn.model_selection import train_test_split #Paqueterias para entrenamiento de modelos
from tqdm import tqdm

import numpy as np #Nos ofrece funciones matematicas
import pandas as pd #Libreria especializada en el analisis y manipulación de datos
import matplotlib.pyplot as plt  #Visualización de las imagenes
import albumentations as A
import numpy as np


###Cargar el modelo ZIP y el TOKEN del bot

In [ ]:
loaded_model = torch.jit.load('model.zip')
TOKEN = "6699114144:AAG0lOR47iOi098QgSLQezHI_FcH02msZZo"   # Este es para comunicarse con el bot

## Código del modelo

Clases del preprocesamiento y postprocesamiento del modelo

In [ ]:
class Preprocessing():
  def __init__(self): #Comenzamos con la función de inicialización de datos
    super().__init__()
    #self.file_paths = file_paths

    self.augmentations = A.Compose([
        A.Resize(320, 320)
        ])

  # def __len__(self): #Longitud de datos para cada set
  #   return len(self.file_paths)

  def __getitem__(self,imagen): #Selección de cada dato

    # file_path = self.file_paths[idx] #Seleccionamos la columna de la base



    image = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB) #Pasamos al canal RGB



    if self.augmentations: #Funciones para la visión artificial
      data = self.augmentations(image = image)
      image = data['image'] #Selección de la imagen a partir de los datos desplegados anteriormente

    image = np.transpose(image, (2,0,1)).astype(np.float32) #Ajustamos las caracteristicas de la imagen
    image = torch.Tensor(image) / 255.0 #Usamos torch para la definición de la arquitectura


    return  image

In [ ]:
class Postprocessing():
  def __init__(self): #Comenzamos con la función de inicialización de datos
    #self.logits_mask = logits_mask
    #self.image = image
    super().__init__()

  def postprocess(self, logits_mask, image):
    pred_mask = torch.sigmoid(logits_mask)
    pred_mask = (pred_mask > 0.9)*1.0
    pred_mask = pred_mask.cpu().squeeze(0).squeeze(0).T

    x = image.T
    #ymul = pred_mask.unsqueeze(2)
    y=pred_mask
    #z = np.multiply(x, ymul)
    #return z.numpy()
    return y.numpy()

  def postprocessmul(self, logits_mask, image):
    pred_mask = torch.sigmoid(logits_mask)
    pred_mask = (pred_mask > 0.9)*1.0
    pred_mask = pred_mask.cpu().squeeze(0).squeeze(0).T

    x = image.T
    ymul = pred_mask.unsqueeze(2)
    y=pred_mask    ### --- MDF
    z = np.multiply(x, ymul)
    #return z.numpy()
    return  y.numpy(), z.numpy()    ### ---- MDF

In [ ]:
albumentations_dataset = Preprocessing()

In [ ]:
def segmentar(imp):
  """esta función recibe la imagen y la regresa segmentada"""
  imagencul=albumentations_dataset.__getitem__(imp)
  logits_mask = loaded_model(imagencul.unsqueeze(0)) ########
  imagencul_post = Postprocessing()

  y, x=imagencul_post.postprocessmul(logits_mask,imagencul)   ### --- MDF

  imagen = Image.fromarray((x*255).astype('uint8'))
  imagen = imagen.transpose(Image.FLIP_LEFT_RIGHT)
  imagen = imagen.transpose(Image.Transpose.ROTATE_90)

  mascara = Image.fromarray((y*255).astype('uint8'))    #
  mascara = mascara.transpose(Image.FLIP_LEFT_RIGHT)        # --- MDF
  mascara = mascara.transpose(Image.Transpose.ROTATE_90)  #

  return imagen, mascara    #### MDF

El otro modelo:

In [ ]:
import joblib
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
bkm = joblib.load('regression.joblib') # Carga del modelo.    ### Este se comenta MDF

In [ ]:
def getlabelimg(msked_img, msk, model):
    flat_img=flatten_img(msked_img)
    ypred = model.predict(flat_img).astype(float)
    img_pred = ypred.reshape([msked_img.shape[0],msked_img.shape[1]])
    if len(msk.shape)==3:
        msk = msk[:,:,0]
        msk = msk==0
    img_pred[msk] = np.nan
    return img_pred



In [ ]:
def drop_background(msked_img): #without background
    masked_flat_img = flatten_img(msked_img)
    region_msk = (masked_flat_img == masked_flat_img[0]).sum(axis=1) == 3
    masked_flat_img[region_msk]=np.nan
    masked_img=masked_flat_img.reshape([msked_img.shape[0],msked_img.shape[1],3])
    return masked_img

In [ ]:
def flatten_img(masked_img):
    img_flat = masked_img.reshape([masked_img.shape[0]*masked_img.shape[1],3])
    img_flat = img_flat.astype(np.float64)
    img_flat = img_flat[~np.isnan(img_flat[:,0])]
    return img_flat

In [ ]:
from matplotlib import cm
color = cm.get_cmap('jet',3)

<ipython-input-19-b4eb7a50bd55>:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  color = cm.get_cmap('jet',3)


----------

## Código del bot

In [ ]:
bot = telebot.TeleBot(TOKEN)  # instanciamos el bot

In [ ]:
# Para que responda al comando: /start y /ayuda    (esto solo es prueba y por si es útil después)
@bot.message_handler(commands=["start"])   # para /start
def bienvenida_start(message):
    """Da la bienvenida al usuario del bot"""
    bot.reply_to(message, "Bienvenido/a, envía tu primer imagen para procesarla:")
#     print(dir(message))  (Para ver los atributos de message)

@bot.message_handler(commands=["ayuda"])    # para /ayuda
def mensaje_ayuda(message):
    """Da un mensaje de sección de ayuda"""
    bot.reply_to(message, "Esta es la sección de ayuda")

In [ ]:
# Para responder a otras cosas (en este caso solo para texto y fotos):

@bot.message_handler(content_types=["text", "photo"])  # para otras cosas, se agrega la otra cosa (audio, video, etc.) y se modifica el código
def bot_mensajes_texto(message):
    """Gestiona los mensajes de texto"""
    if message.text and message.text.startswith("/"):  # en caso de que empiece como comando o no haya texto en el mensaje ( que sea un archivo u otra cosa que no sea imagen)
        bot.send_message(message.chat.id, "Comando no disponible")
    else:
        foto_id = message.photo[-1].file_id # id de la imagen original
        foto_info = bot.get_file(foto_id)  # obtenemos más info de la imagen
        foto_path = foto_info.file_path # obtenemos el path de la imagen
        image_url = f'https://api.telegram.org/file/bot{TOKEN}/{foto_path}'   # URL completo de la imagen

        try:
            response = requests.get(image_url)
            response.raise_for_status()  # Verifica si la solicitud fue exitosa
            imagen = Image.open(BytesIO(response.content))   # Lee la imagen desde response y la convierte a un objeto Image
            bot.send_chat_action(message.chat.id, "upload_photo")
          ### Modificamos la imagen -----
            #img = np.array(imagen) # Aquí la convertimos a array por si así la toma el modelo:
            img =cv2.cvtColor(np.array(imagen), cv2.COLOR_RGB2BGR)

            imagen_segmentada, mascara0 = segmentar(img)  ### --- MDF

            mascara0.save("mascara0.png")   ### --- MDF

            #print(np.array(img))
            #imagen_modificada = imagen.transpose(Image.Transpose.ROTATE_90) # Transponemos la imagen 90 grados.
          ### --------------------------

        ## Guardamos la imagen modificada:
            imagen_segmentada.save("segmented_image.png")

            # ======= MDF --- Voy a volver a cargar las imágenes como array:

            img_seg = plt.imread("segmented_image.png")   ### --- MDF
            # mask_0 = plt.imread("mascara0.png")   ### --- MDF

            msk3=cv2.imread("mascara0.png",cv2.IMREAD_GRAYSCALE) ### --- MDF
            cimage=cv2.cvtColor(msk3,cv2.COLOR_GRAY2BGR) ### --- MDF

            img_clasificada = getlabelimg(img_seg,cimage,bkm) ### --- MDF

            plt.title('Clasificación por tejidos')
            plt.imshow(img_clasificada,cmap=color, vmin=0, vmax=2)
            plt.colorbar(ticks=[])
            # plt.xlim(0,70)
            # plt.ylim(0,90)
            plt.xticks(range(0,100,100))
            plt.yticks(range(0,100,100))
            plt.savefig("clasif.png")
            # plt.show()

            # ========  MDF -------


        # Devolvemos la imagen:
            with open("segmented_image.png", "rb") as photo:
            #with open("clasif.png", "rb") as photo:
                bot.send_photo(message.chat.id, photo, caption = "Esta es la imagen *segmentada*", parse_mode = "MarkdownV2")

        #### --- MDF ---
            with open("clasif.png", "rb") as photo:
                bot.send_photo(message.chat.id, photo, caption = "Esta es la imagen *clasificada* donde los tejidos son Rojo epitelial, Verde esfacelar y Azul necrosado", parse_mode = "MarkdownV2")
        ### -- MDF ..

        except:
            print("Error al obtener la imagen desde la URL")
            bot.send_message(message.chat.id, "Hubo un error al obtener la imagen desde la URL.")



In [ ]:
def recibir_mensajes():    # Esta función es solo para hacer el hilo.
  """Bucle infinito que comprueba si hay nuevos mensajes en el bot"""
  bot.infinity_polling()

## Código que corre el bot

Corremos la siguiente celda para que entre en el bucle:
(Hay que reiniciar el kernel para volver a correrlo)

In [ ]:
# MAIN code-------------------
if __name__ == '__main__':
    print('Iniciando el bot ...')
    hilo_bot = threading.Thread(name="hilo_bot", target = recibir_mensajes)
    hilo_bot.start()
    print('Bot iniciado, el bot está funcionando')

Iniciando el bot ...
Bot iniciado, el bot está funcionando
